In [21]:
import pandas as pd

path_train = 'C:/Data/train.csv'
path_test = 'C:/Data/test.csv'

train = pd.read_csv(path_train, index_col='PassengerId') 
test = pd.read_csv(path_test, index_col='PassengerId') 

y_train = train['Survived']
train_sep = train.drop(columns=['Survived'])

In [22]:
def encode_categorical(df_train, df_test):
    
    test_start_ind = df_train.index[-1] 
    
    concated_df = pd.concat([df_train, df_test], sort=False) 
        
    encoded_df = pd.get_dummies(concated_df, drop_first=True)
    
    
    df_train_encoded, df_test_encoded = encoded_df.iloc[:test_start_ind, :], encoded_df.iloc[test_start_ind:, :] 
    
    return df_train_encoded, df_test_encoded

In [ ]:
X_train, X_test = encode_categorical(train_sep, test)

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier()

parameters = {
              'criterion': ['entropy', 'gini'],
              'max_depth': range(1, 10),
              'min_samples_split': range(2, 10),
              'min_samples_leaf': range(1, 10)
             }


grid_search_cv = GridSearchCV(clf, parameters, cv=5)

In [24]:
%%time

grid_search_cv.fit(X_train, y_train)

CPU times: total: 5min 6s
Wall time: 5min 6s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 10),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10)})

In [25]:
model = grid_search_cv.best_estimator_
grid_pred = model.predict(X_test)

In [26]:
def create_a_file(prediction, test_data, name):
    
    result = pd.DataFrame({'PassengerId': test_data.reset_index()['PassengerId'], "Survived": prediction})  
    result.to_csv(name, index=False)

In [27]:
create_a_file(grid_pred, test, 'titanic_dt_gridcv.csv')

<img src="ScoreTitanic.png" alt="Підпис_до_зображення">
